In [1]:
import pandas as pd
import numpy as np

input_file = './dataSet_phase2/trajectories(table_5)_training2.csv'
data = pd.read_csv(input_file)
data

,intersection_id,tollgate_id,vehicle_id,starting_time,travel_seq,travel_time
0,B,3,1032458,2016-10-18 00:02:32,105#2016-10-18 00:02:32#8.51;100#2016-10-18 00...,47.55
1,C,1,1008361,2016-10-18 00:06:19,115#2016-10-18 00:06:19#8.45;102#2016-10-18 00...,144.57
2,B,3,1030608,2016-10-18 00:07:11,105#2016-10-18 00:07:11#6.00;100#2016-10-18 00...,81.84
3,A,2,1045369,2016-10-18 00:14:07,110#2016-10-18 00:14:07#8.74;123#2016-10-18 00...,32.69
4,A,3,1028919,2016-10-18 00:14:21,110#2016-10-18 00:14:21#7.84;123#2016-10-18 00...,76.52
...,...,...,...,...,...,...
10131,A,3,1027137,2016-10-24 23:42:15,110#2016-10-24 23:42:15#8.08;123#2016-10-24 23...,102.61
10132,B,3,1018560,2016-10-24 23:42:44,105#2016-10-24 23:42:44#7.19;100#2016-10-24 23...,94.89
10133,A,3,1049703,2016-10-24 23:44:09,110#2016-10-24 23:44:09#13.03;123#2016-10-24 2...,112.09
10134,B,3,1016550,2016-10-24 23:50:31,105#2016-10-24 23:50:31#9.67;100#2016-10-24 23...,105.62


In [2]:
def getTimeTange(time):
    # 取时间窗口
    left_time = time.replace(minute=(int)(time.minute / 20) * 20, second=0)
    return str(left_time).split(' ')[1]

In [11]:
data['starting_time']=pd.to_datetime(data['starting_time'])
data['date'] = data['starting_time'].dt.date
data['hour'] = data['starting_time'].dt.hour
data['time'] = data['starting_time'].map(lambda x: getTimeTange(x))

In [12]:
trajectories_train = data.drop('travel_seq',axis=1)

In [13]:
first_2hours = [8, 10, 18, 19] 
ind = (trajectories_train['hour'] >= first_2hours[0]) & (trajectories_train['hour'] < first_2hours[1])
ind1 = (trajectories_train['hour'] >= first_2hours[2]) & (trajectories_train['hour'] < first_2hours[3])
ind2 = ind | ind1
trajectory_train = trajectories_train[ind2]

In [22]:
trajectory_train['date'] = trajectory_train['date'].astype(str)
trajectory_train['time'] = trajectory_train['time'].astype(str)
trajectory_train['starting_time'] = trajectory_train['date']+ ' '+trajectory_train['time']

/Users/liyuan/anaconda2/envs/py3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/Users/liyuan/anaconda2/envs/py3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/liyuan/anaconda2/envs/py3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_inde

In [30]:
trajectory_train=trajectory_train[['intersection_id','tollgate_id','starting_time','travel_time']]
ground_truth = trajectory_train.groupby(['intersection_id','tollgate_id','starting_time'])['travel_time'].mean().reset_index(
        name='mean_link_time')

def getTimeWindow(start_time):
    time = pd.to_datetime(start_time)
    return '['+str(time)+','+str(time + pd.Timedelta(minutes=20))+')'


In [34]:
4*3*7*6 #4小时，每小时3段，7天，6车道

504

In [41]:
#读自己预测的
xgb = pd.read_csv('submission_sample/xgb_travelTime.csv')
xgb['starting_time'] = xgb['time_window'].apply(lambda x: x.split(',')[0][1:])
xgb

,intersection_id,tollgate_id,time_window,avg_travel_time,starting_time
0,A,2,"[2016-10-18 08:00:00,2016-10-18 08:20:00)",65.410620,2016-10-18 08:00:00
1,A,2,"[2016-10-18 08:20:00,2016-10-18 08:40:00)",69.239240,2016-10-18 08:20:00
2,A,2,"[2016-10-18 08:40:00,2016-10-18 09:00:00)",68.185750,2016-10-18 08:40:00
3,A,2,"[2016-10-18 09:00:00,2016-10-18 09:20:00)",67.538956,2016-10-18 09:00:00
4,A,2,"[2016-10-18 09:20:00,2016-10-18 09:40:00)",68.319660,2016-10-18 09:20:00
...,...,...,...,...,...
499,C,3,"[2016-10-24 17:20:00,2016-10-24 17:40:00)",156.848430,2016-10-24 17:20:00
500,C,3,"[2016-10-24 17:40:00,2016-10-24 18:00:00)",160.687740,2016-10-24 17:40:00
501,C,3,"[2016-10-24 18:00:00,2016-10-24 18:20:00)",159.357100,2016-10-24 18:00:00
502,C,3,"[2016-10-24 18:20:00,2016-10-24 18:40:00)",163.905430,2016-10-24 18:20:00


In [49]:
merged = pd.merge(ground_truth,xgb,on= ['intersection_id','tollgate_id','starting_time'],how='left')
merged = merged.dropna(axis=0,how='any')
merged.apply(lambda x: abs(x['mean_link_time'] - x['avg_travel_time'])/ x['mean_link_time'], axis=1).mean()

0.2176157673341964